In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# session_handler
> A class to handle the session of the user.

In [ ]:
#| default_exp session_handler

In [ ]:
class AppStart:
  pass
  

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()